# PREPROCESSAMENTO
Esse arquivo define funções de extração de características. Preferẽncia para o uso de léxicos

### Léxico de Cláudia Freitas

Uso do léxico encontrado <a href = http://www.scielo.br/pdf/rbla/v13n4/aop2813.pdf>neste artigo</a>.
Peguei o léxico de substantivos desse artigo (por enquanto) e salvei num arquivo txt. É necessário uma expressão regular para extrair as palavras do arquivo. 

As palavras encontram-se na seguinte forma: [word];

In [93]:
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from database_utils import DatabaseConnector, build_dataframe, normalize_text
from sklearn.cross_validation import StratifiedKFold

def extract_words(filename):
    import re
                       
    f = open(filename, 'r')
    file_content = ''.join(f.readlines())
    f.close()
    
    words = re.findall(r'\[(\w+)\]', file_content)
    return words

def get_data_from_db(sentiment = None):
    db_connector = DatabaseConnector('localhost', 'root', '12345', 'CORPUS_VIES')
    if sentiment != None:
        retrieved_data = db_connector.getDataBySentiment(sentiment)
    else:
        retrieved_data = db_connector.getDataTextAndLabel()

    return retrieved_data


# Recuperação dos textos - treinar o modelo somente com as classes POS e NEG
all_data = get_data_from_db()
all_data = all_data[(all_data["labels"] == "PO") | (all_data["labels"] == "NG")]
print('Textos carregados')

subs_pos = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/subs_pos')
subs_neg = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/subs_neg')
verbs_pos = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/verbs_pos')
verbs_neg = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/verbs_neg')
adj_pos = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/adj_pos')
adj_neg = extract_words('/home/caiomagno/Documentos/Analise de sentimento - aplicacoes/Recursos/Claudia Freitas/adj_neg')
print('Léxico carregado')

Textos carregados
Léxico carregado


In [112]:
# Vetorizando os textos do corpus viés através do léxico de Cláudia Freitas
lexicon = list(set(subs_neg + subs_pos + verbs_neg +verbs_pos + adj_neg + adj_pos))
ftu = FeatureUnion([("unigram", CountVectorizer()), ("bigram", CountVectorizer(ngram_range=(1,2))), ("lexicon_vector", CountVectorizer(vocabulary= lexicon))])
print('Extração de features realizadas\n')

sKFold = StratifiedKFold(all_data['labels'], n_folds=10, shuffle=True)

print("Cross Validation:")
accuracy_average = np.array([])
for index, (train, test) in enumerate(sKFold):
    # Treinando um modelo Naive Bayes
    train_data = all_data.iloc[train]
    test_data = all_data.iloc[test]
    
    p = Pipeline([("features", ftu), ("classifier", MultinomialNB())])
    p.fit(train_data["texts"], train_data["labels"])
    predictions = p.predict(test_data["texts"])

    accuracy = accuracy_score(test_data["labels"], predictions, normalize=True)
    accuracy_average = np.append(accuracy_average, accuracy)
    print("Fold ", index, " - Acuracia: ", accuracy)
    

print("\nAccuracia media: ", accuracy_average.mean())
print("Desvio padrão: ", accuracy_average.std())

Extração de features realizadas

Cross Validation:
Fold  0  - Acuracia:  0.732394366197
Fold  1  - Acuracia:  0.757142857143
Fold  2  - Acuracia:  0.8
Fold  3  - Acuracia:  0.757142857143
Fold  4  - Acuracia:  0.814285714286
Fold  5  - Acuracia:  0.728571428571
Fold  6  - Acuracia:  0.814285714286
Fold  7  - Acuracia:  0.7
Fold  8  - Acuracia:  0.657142857143
Fold  9  - Acuracia:  0.628571428571

Accuracia media:  0.738953722334
Desvio padrão:  0.0602358974283


Analisando a matriz de features com PCA

In [125]:
from sklearn.decomposition import PCA

lexicon = list(set(subs_neg + subs_pos + verbs_neg +verbs_pos + adj_neg + adj_pos))
ftu = FeatureUnion([("unigram", CountVectorizer()), ("bigram", CountVectorizer(ngram_range=(1,2))), ("lexicon_vector", CountVectorizer(vocabulary= lexicon))])

cv_unigram = CountVectorizer()
cv_bigram = CountVectorizer()

unigram = 

print("Dimensionalidade do problema")
print("Dimensão do léxico: ", len(lexicon))
print("Dimensões da matriz: ", bag_of_features.shape)

bag_of_

Dimensionalidade do problema
Dimensão do léxico:  513
Dimensões da matriz:  (2, 517)
